In [1]:
import numpy as np
import random
import math

In [2]:
class LinearPolarizer:
    def __init__(self):
        
        self.x = np.array([[1], [0]])
        self.y = np.array([[0], [1]])
    
    def horizontal_vertical(self, bit):
        if bit == 0:
            return self.x
        else:
            return self.y
    
    def diagonal_polarization(self, bit):
        jones = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])
        
        if bit == 0:
            return np.dot(jones, self.x)
        else:
            return np.dot(jones, self.y)
    
    def general_polarization(self, angle, basis):
        """
            angle to be in degrees
        """
        angle = (math.pi/180) * (angle)
        jones = np.array([[np.cos(angle), np.sin(angle)], [np.sin(angle), -np.cos(angle)]])
        
        return np.dot(jones, basis)

class PolarizingBeamSplitter:
    def __init__(self):
        pass
    
    def measure(self, vector, basis):
        """
            basis  : basis chosen by bob to measure polarization encoded photon
                        0 -> horizontal/vertical 
                        1 -> diagonal
            vector : Jones vector for polarized photon
            
            returns a dictionary with probabilities of the encoded bit sent by Alice being 0 or 1
        """
        #horizontal-vertical can be clubbed into an identity matrix
        horizontal = np.array([[1, 0], [0, 0]])
        vertical = np.array([[0, 0], [0, 1]])
        plus_minus = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])

        if basis == 0:
            zero = np.dot(horizontal, vector)[0]
            one = np.dot(vertical, vector)[1]
        
        elif basis == 1:
            zero = np.dot(plus_minus, vector)[0]
            one = np.dot(plus_minus, vector)[1]
        else:
            return None
            
        return {0: zero[0]**2, 1: one[0]**2}

In [3]:
class Alice:
    def __init__(self, n):
        self.n = n
        self.alice = {} #{no. : [bit encoded, basis chosen to encode in ]} no. is some unique number
                        #{1:[0,0],2:[0,1],3:[0,0]} Example

    def generate_and_encode(self): 
        """
            Will generate n bits randomly
            For each bit generated, a basis is chosen in which it is encoded
            Dependency for encoding: <class LinearPolarizer>
                0-> horizontal/vertical polarization
                1-> diagonal polarization
            
            Should generate a dictionary of the form self.alice mentioned above
        """
        LP = LinearPolarizer()
        encode = []
        count = self.n
        
        while count!= 0:
            self.alice[count] = [ random.randint(0,1), random.randint(0,1)]
            if self.alice[count][1] == 0:
                encode.append(LP.horizontal_vertical(self.alice[count][0]))
            else:
                encode.append(LP.diagonal_polarization(self.alice[count][0]))
            count-=1
        
        return encode
    

class Bob:
    def __init__(self, n):
        self.n = n
        self.bob = {} #{no. : [bit after measurement, basis chosen to measure in]}
                      #{1:[1,0],2:[0,0],3:[1,0]} Example
    
    def choose_basis_and_measure(self, received):
        """
            received : the data received by bob
            Dependency for measurement: <class PolarizingBeamSplitter>
            
                0-> horizontal/vertical polarization
                1-> diagonal polarization
            
            Should generate a dictionary of the form self.bob mentioned above
        """
        #self.bob[n][0] is the measured bit
        
        PBS = PolarizingBeamSplitter()
        count = self.n
        
        while count!= 0:
            i = count
            self.bob[count] = [0, random.randint(0,1)]
            self.bob[count][0] = random.randint(0,1) if (PBS.measure(received[i-count], self.bob[count][1])[0] == PBS.measure(received[i-count], self.bob[count][1])[1]) else (0 if (PBS.measure(received[i-count], self.bob[count][1])[0]> PBS.measure(received[i-count], self.bob[count][1])[1]) else 1)
            #Here,  picking randomly between 0 and 1 if wrong basis is choosen else 0 or 1 based of actual measurement
            count-=1

### Errors are caused by interference from Eve and noise in the channel
### Assumption: All errors are assumed to be caused by Eve   

    1. Alice generates a bit string dof (4+delta)n random bits
    2. Alice randomly chooses a basis to encoded each bit of her bit string and encodes it
        - Basis chosen and the encoded bit is kept track of 
    3. Alice sends the resulting encoded bits to Bob
    4. Bob recieves (4+delta)n bits from Alice (probably erroneous)
        - Bob then radnomly chooses a basis for each of these bits and measures in that basis
        - Basis chosen for measurement and the result of measurement are both kept track of
    5. Alice and Bob do basis reconciliation 
        - With high probability there are still 2n bits remaining, if not abort the protocol
    6. Alice randomly samples 2n bits from the remaining bits (>=2n) and announces which bits she picked
        (but not their values)
    7. Alice randomly selects n bits from these 2n bits as "check bits" and announces them AND their values
    8. Bob compares the bit values he measured for the n check bits selected by Alice and 
        announces the bits where they disagree
        - If more than an acceptable numberof these check bit values disagree, they abort the protocol 
    9. Alice now has an nbit string x, and Bob has an n-bit string x+ e1, 
        where e1 is the error caused by Eve’s interference and/or channel noise
    10. Error correction is done due to which Alice and Bob have the same keys
    11. Privacy amplification

In [4]:
class BB84:
    def __init__(self, n, delta, error_threshold_1, error_threshold_2):
        """
            Alice generates (4+delta)n bits 
            delta: small fraction less than one 
            error_threshold_1: if error is greater than this, key generation is aborted
            error_threshold_2: if error while announcing n bits from 2n bits is greater than this
                                key generation is aborted 
        """
        if delta > 1:
            print("Value for delta should be lesser than 1")
            return 
        
        self.total = math.ceil(4 + delta)*n
        self.alice = Alice(self.total)
        self.bob = Bob(self.total)
        
        self.error1 = error_threshold_1
        self.error2 = error_threshold_2